In [1]:
import numpy as np
import pandas as pd
import spacy
import re
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
cv = CountVectorizer()

df = pd.read_csv('Crowd_mar_apr_TOT_2.csv', sep=';', keep_default_na=False, low_memory=False, error_bad_lines=False)

#creo una colonna che contiene tutte le parti di testo analizzabili

#df['complessiva']=df['Message'].astype(str)+' ' +df['Image Text'].astype(str)+' '+df['Link Text'].astype(str)+ ' '+df['Description'].astype(str)
df['Tot'] = df['Totale'].astype(str)

nlp = spacy.load('it_core_news_lg')
nlp_en = spacy.load('en_core_web_lg')
nlp_es = spacy.load('es_core_news_lg')
nlp_fr = spacy.load('fr_core_news_lg')
nlp_de = spacy.load('de_core_news_lg')


#metodo per rimuovere siboli e emoticons
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u000D" #ritorno a capo
                               u"\u000A" #newline
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               u"\u0023" #cancelletto
                               u"\u0030-\u0039"#numeri
                               u"\u00A0-\u00BB" #rimozione virgolette strane
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

#metodo non usato al momento
def tokenize(texts):
    """
    Tokenizes a list of texts, returning a list of tokens.
    """
    token_texts = []
    for text in texts:
        token_texts.append(simple_preprocess(text, deacc=True))
    return token_texts

#metodo per creare i lemmi, contiene una lista di stopword ulteriore
def lemming_cell(text):
    stopword=['nn','settembrese','x','xke','codvid','di','a','da','in','con','contro','coi','su',
              'sui','sullo','sulle','sulla','sugli','sull','per','tra','fra','ho','sono','ad','allo','agli',
              'alla','alle','allo','all','col','dal','dei','dallo','dalla','dalle','dagli','dell','il','lo',
              'la','i','gli','le','un','uno','una','io','tu','lui','lei','noi','voi','loro','mio','mia',
              'miei','mie','tuo','tua','tuoi','tue','suo','sua','suoi','già','gia','eh','cè','ce','poi','boh',
              'sue','nostro','nostra','nostri','nostre','vostro','vostra','vostri','vostre','mi','ti','ci',
              'vi','ma','ed','se','perché','anche','come','dov','dove','che','chi','cui','non','più','quale',
              'quanto','quanti','quanta','quante','quello','quelli','quella','quelle','questo','questi',
              'questa','queste','si','no','non','tutto','tutti','a','c','e','i','l','o','ho','hai','ha',
              'abbiamo','avete','hanno','abbia','abbiate','abbiano','avere','avrò','avro','avrai','avrà',
              'avremo','avrete','avranno','avrei','avresti','avrebbe','avremmo','avreste','avrebbero','avevo',
              'avevi','aveva','avevamo','avevate','avevano','ebbi','avesti','ebbe','avemmo','aveste','ebbero',
              'avessi','avesse','avessimo','avessero','avendo','avuto','avuta','avuti','avute','essere','sono',
              'sei','è','siamo','siete','sia','siate','siano','sarò','saro','sarai','sarà','sara','saremo',
              'sarete','saranno','sarei','saresti','sarebbe','saremmo','sareste','sarebbero','ero','eri','era',
              'eravamo','eravate','erano','fui','fosti','fu','fummo','foste','furono','fossi','fosse','fossimo',
              'fossero','essendo','faccio','fai','facciamo','fanno','faccia','facciate','facciano','farò',
              'farai','farà','fara','faremo','farete','faranno','farei','faresti','farebbe','faremmo',
              'fareste','farebbero','facevo','facevi','faceva','facevamo','facevate','facevano','feci',
              'facesti','fece','facemmo','faceste','fecero','facessi','facesse','facessimo','facessero',
              'facendo','sto','stai','sta','stiamo','stanno','stia','stiate','stiano','starò','starai','starà',
              'staremo','starete','staranno','starei','staresti','starebbe','staremmo','stareste','starebbero',
              'stavo','stavi','stava','stavamo','stavate','stavano','stetti','stesti','stette','stemmo',
              'steste','stettero','stessi','stesse','stessimo','stessero','stando','dare','diedi','dai',
              'daremo','darete','davo','davi','dava','davamo','davate','davano','dasti','dassero','dando',
              'davasti','desti','davammo','davaste','diedero','leggere','letto','leggo','leggiamo','photos',
              'leggete','leggono','leggevo','leggeva','leggevi','leggevamo','leggevate','leggevano','leggendo',
              'mantenere','manteniamo','mantengo','mantieni','mantiene','mantenete','mantengono','mantenevo',
              'mantenevi','manteneva','mantenuto','mantenendo','rispettare','rispetto','rispetti','rispetta',
              'rispettiamo','rispettate','rispettano','rispettavo','rispettavi','rispettava','rispettato',
              'intervistare','intervistato','intervistavano','andare','andavo','andato','andai','vado','vai',
              'va','andiamo','andate','vanno','andando','andava','andavi','arrivare','numerare','mettere',
              'messo','metto','sapere','fino','così','cosi','bho']
    
    stopword_en = nlp_en.Defaults.stop_words
    stopword_es = nlp_es.Defaults.stop_words
    stopword_fr = nlp_fr.Defaults.stop_words
    stopword_de = nlp_de.Defaults.stop_words
    
    tokens=[] 
    doc=nlp(text.lower()) #trasforma tutte le parole in minuscolo in minuscolo
    
    for token in doc:
        # se la parola non è un URL e non è nella lista stop word, allora mette il lemma nel vettore "tokens"
        if token.pos_ in ['PROPN','NOUN','VERB','ADJ','ADV'] and not token.is_stop and not token.like_url and token.lemma_ not in stopword and token.lemma_ not in stopword_en and token.lemma_ not in stopword_es and token.lemma_ not in stopword_fr and token.lemma_ not in stopword_de:
            tokens.append(token.lemma_)
    tokens=" ".join(set(tokens)) # trasforma vettore tokens in una stringa
    return tokens
"""
#costruisco un file con le colonne riunite ma ripulite per TLAB
finale=open("dati.csv","w")
for i in df['complessiva']:
    if len(i)<2000:
        pulito=remove_emoji(str(i))
        a=pulito.replace('  ',' ')
        a=a.replace('\n','')
        finale.write(a)
        finale.write('\n')
        
finale.close() 
"""
#costruisco un corpus con i singoli post già puliti da emoticon, da segni di interpunzione e url

corpus=[] # corpus è un vettore di stringhe "pulite"
for i in df['Tot']:
    phrase=remove_emoji(str(i))
    vett=lemming_cell(phrase)
    if vett !="":
        corpus.append(vett)

FileNotFoundError: [Errno 2] No such file or directory: 'Crowd_mar_apr_TOT_2.csv'

In [5]:
dataset = pd.DataFrame(corpus)
dataset.to_excel("Dataset_TLAB.xlsx")

In [3]:
# Costruzione dizionario per il topic

dtm = cv.fit_transform(corpus)
LDA = LatentDirichletAllocation(n_components=5,n_jobs=-1) # funzione per specificare il numero di cluster
LDA.fit(dtm)


#len(cv.get_feature_names())
for index,topic in enumerate(LDA.components_):
    print(f'TOP 30 PAROLE DEL CLUSTER #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-30:]])
    print('\n')

TOP 30 PAROLE DEL CLUSTER #0
['parlare', 'dottore', 'sanità', 'mascherina', 'grande', 'materiale', 'lavorare', 'combattere', 'infermiere', 'virus', 'epidemia', 'cina', 'bergamo', 'milano', 'lineare', 'cinese', 'personale', 'lottare', 'aiutare', 'lombardia', 'esperto', 'covid', 'italiano', 'emergenza', 'sanitario', 'ospedale', 'italia', 'medico', 'medicare', 'coronavirus']


TOP 30 PAROLE DEL CLUSTER #1
['malato', 'lavorare', 'ricoverato', 'guarire', 'intensivo', 'morto', 'ricoverare', 'napoli', 'curare', 'famiglia', 'sindacare', 'caso', 'terapia', 'dottore', 'reparto', 'risultare', 'emergenza', 'notizia', 'personale', 'sanitario', 'morire', 'tampone', 'medico', 'contagiare', 'paziente', 'positivo', 'covid', 'ospedale', 'coronavirus', 'medicare']


TOP 30 PAROLE DEL CLUSTER #2
['epidemiologo', 'settimana', 'possibile', 'caso', 'virologa', 'nuovo', 'parola', 'studiare', 'rischiare', 'notizia', 'burioni', 'rispondere', 'scientifico', 'università', 'dato', 'fase', 'italiano', 'spiegare', '

In [4]:
# Creazione grafico

from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis
import os
number_topics=5
count_data = dtm

A=len(cv.get_feature_names())

LDAvis_data_filepath = os.path.join('./ldavis_prepared2_3_'+str(number_topics))

# this is a bit time consuming - make the if statement True
# if you want to execute visualization prep yourself

if 1 == 1:
    LDAvis_prepared = sklearn_lda.prepare(LDA, dtm, cv, mds='mmds')
    #print (LDAvis_prepared)
with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared,f,0)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared2_3_'+ str(number_topics) +'.html')

count_data = cv.fit_transform(df['Tot'])
print(count_data)

  (0, 10088)	1
  (0, 42839)	1
  (0, 57905)	1
  (0, 26250)	2
  (0, 55122)	1
  (0, 76654)	2
  (0, 62152)	2
  (0, 68055)	1
  (0, 71972)	1
  (0, 41919)	4
  (0, 25117)	5
  (0, 1043)	5
  (0, 80571)	1
  (0, 68197)	1
  (0, 44096)	2
  (0, 9910)	1
  (0, 56725)	1
  (0, 23831)	1
  (0, 31127)	1
  (0, 33789)	1
  (0, 19216)	3
  (0, 80245)	1
  (0, 40594)	1
  (0, 64655)	1
  (0, 70869)	1
  :	:
  (59394, 75232)	1
  (59394, 91690)	1
  (59394, 64374)	1
  (59394, 56809)	1
  (59394, 59312)	2
  (59394, 21834)	1
  (59394, 55113)	1
  (59394, 79619)	1
  (59394, 42443)	1
  (59394, 80322)	2
  (59394, 89293)	1
  (59394, 67760)	1
  (59394, 50705)	1
  (59394, 30755)	1
  (59394, 8369)	1
  (59394, 60641)	2
  (59394, 42585)	1
  (59394, 65503)	1
  (59394, 90707)	1
  (59394, 62352)	1
  (59394, 50412)	1
  (59394, 63538)	1
  (59394, 61340)	1
  (59394, 77424)	1
  (59394, 20182)	1


In [ ]:

# Show graph
x = range(start, limit, step)
plt.figure(figsize=(15, 10))
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
plt.show()